### What Exactly is a "Chain" in LangChain?

In the simplest terms, a Chain is a sequence of calls. Think of it as an assembly line for your AI application. Instead of just one worker (a single LLM call), you have a series of workers, each performing a specific task in a set order. This allows you to build more sophisticated and coherent applications by combining multiple components.

Imagine you're building an app to help you plan a trip. A single LLM call might give you a generic itinerary. But with a Chain, you could:

Step 1: Take the user's destination and find the top attractions (first LLM call).

Step 2: Take that list of attractions and find the best local restaurants near them (second LLM call, using the output from step 1).

Step 3: Take the restaurant list and generate a sample dinner menu (third LLM call).

See how the output from one step seamlessly becomes the input for the next? That’s the magic of Chains! They allow you to break down a complex task into smaller, manageable steps.

## Simple Chain

In [3]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

model = GoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

print(result)

chain.get_graph().print_ascii()

Here are 5 interesting facts about cricket:

1.  **Test Matches Can Last Up to Five Days (and Still Be a Draw):** Unlike most other major sports, a single Test match in cricket is designed to be played over five consecutive days, with each team batting twice. Even after all that time, the game can still end in a draw if a winner isn't decided by the end of the fifth day!

2.  **Sir Donald Bradman's Unbreakable Record:** Australian legend Sir Donald Bradman holds the most incredible batting average in Test cricket: **99.94**. To put that in perspective, the next best average for a player with a significant career is typically in the low 60s. He famously needed just 4 runs in his final innings to average 100 but was bowled for a duck.

3.  **A Language of its Own:** Cricket boasts some of the most unique and whimsical terminology in sports. Ever heard of a 'googly,' a 'chinaman,' a 'silly mid-off,' a 'nightwatchman,' a 'ducks' egg,' or a 'sticky wicket'? These are all legitimate terms us

## Sequential Chain

In [4]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

model = GoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in India'})

print(result)

chain.get_graph().print_ascii()

Here's a 5-pointer summary of the provided report on unemployment in India:

1.  **Persistent & Multifaceted Challenge:** Unemployment in India is a complex issue marked by high rates among youth and educated individuals, critically low female labour force participation, and widespread underemployment within the dominant informal sector, indicating a problem of job quality and adequacy, not just joblessness.
2.  **Key Structural Causes:** Major drivers include significant skill mismatches between the workforce and industry demands, slow formal sector job creation despite economic growth, the overwhelming dominance of the low-productivity informal sector, and the immense pressure from a large, continuously growing youth cohort entering the workforce.
3.  **Severe Economic & Social Impacts:** High unemployment leads to substantial economic losses in productivity and GDP, increased fiscal burden on the government, and reduced consumption. Socially, it exacerbates poverty and inequality, c

### Parallel Chain

In [5]:

from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

load_dotenv()

model1 = GoogleGenerativeAI(model="gemini-2.5-flash")

model2 = GoogleGenerativeAI(model="gemini-2.5-flash")

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()

Here is a merged document combining the notes and quiz information about Support Vector Machines:

---

## Support Vector Machines (SVMs): Consolidated Overview

Support Vector Machines (SVMs) are a powerful set of supervised learning methods primarily used for **classification**, **regression**, and **outlier detection**.

### Advantages of SVMs

*   **Effective in High-Dimensional Spaces:** SVMs perform well even when dealing with a large number of features (dimensions), including situations where the number of dimensions is greater than the number of samples.
*   **Memory Efficient:** They achieve memory efficiency by using only a subset of the training points, known as "support vectors," in their decision function.
*   **Versatile Kernel Functions:** SVMs are highly versatile due to their ability to use various "Kernel functions," which can be customized to suit different data types and complex relationships.

### Disadvantages of SVMs

*   **Risk of Overfitting:** There is a risk 

### Conditional Chain

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

model = GoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_f
    
    ormat_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")                    # default Value if no sentiment found
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

That's great! Responding to positive feedback is an excellent opportunity to reinforce good relationships and show appreciation. Here are a few options, ranging from concise to slightly more detailed, choose the one that best fits your context:

**1. Concise & Appreciative:**
*   "Thank you! We really appreciate your positive feedback."
*   "Thanks so much! We're glad to hear you had a positive experience."

**2. Enthusiastic & Encouraging:**
*   "Fantastic! We're absolutely thrilled to hear such positive feedback. It truly motivates us!"
*   "That's wonderful to hear! Thank you for sharing your positive experience."

**3. Slightly More Detailed (Good for customers/clients):**
*   "Thank you so much for your kind words and positive feedback! We're delighted to know you had such a good experience."
*   "We truly appreciate you taking the time to share your positive feedback. It means a lot to us and helps us continue to improve."

**4. Team-Oriented (If applicable):**
*   "Thank you for

: 